In [ ]:
import numpy as np
import scipy.stats as st

**Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины X∼N(μ,σ2) лежит в интервале $\mu\pm c \cdot \sigma$. Чему равно точное значение константы $c$? Округлите ответ до четырёх знаков после десятичной точки.**

In [ ]:
alpha = 1 - 0.997
round(st.norm.ppf(1 - alpha / 2), 4)

**В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.**

**За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.**

**Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.**

In [ ]:
np.round(189 / 11034 - 104 / 11037, 4)

**Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.**

In [ ]:
p1 = 189 / 11034

In [ ]:
p2 = 104 / 11037

In [ ]:
z = st.norm.ppf(1 - .05 / 2)
left = np.round((p1 - p2) - z * np.sqrt((1 - p1) * p1 / 11034 + p2 * (1 - p2) / 11037), 4)
right = np.round((p1 - p2) + z * np.sqrt((1 - p1) * p1 / 11034 + p2 * (1 - p2) / 11037), 4)
print('({:.4f}, {:.4f})'.format(left, right))

**Продолжим анализировать данные эксперимента Гарвардской медицинской школы.**

**Для бернуллиевских случайных величин $X∼Ber(p)$ часто вычисляют величину $\frac{p}{1-p}$, которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо $p$ нужно подставить $\hat{p}$. Например, шансы инфаркта в контрольной группе, принимавшей плацебо, можно оценить как**

$\frac{\frac{189}{11034}}{1-\frac{189}{11034}} = \frac{189}{11034-189}\approx 0.0174$

**Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.**

In [ ]:
plac_odd = 189 / (11034 - 189)
asp_odd = 104 / (11037 - 104)
round(plac_odd, 4), round(asp_odd, 4)

In [ ]:
round(plac_odd / asp_odd, 4)

**Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.**

**Чтобы получить в точности такой же доверительный интервал, как у нас:**

* **составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;**
* **установите random seed=0;**
* **сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.**

In [ ]:
def get_bootstrap_sample(data, size):
    indices = np.random.randint(0, len(data), (size, len(data)))
    sample = np.array(data)[indices]
    return sample

In [ ]:
plac_true, plac_false = 189, 11034 - 189
asp_true, asp_false   = 104, 11037 - 104

plac = [1 for _ in range(plac_true)] + [0 for _ in range(plac_false)]
asp = [1 for _ in range (asp_true)]  + [0 for _ in range(asp_false)]

np.random.seed(0)
plac_bt = get_bootstrap_sample(plac, 1000)
asp_bt = get_bootstrap_sample(asp, 1000)

In [ ]:
from collections import Counter
def get_chance(sample):
    cnt = Counter(sample)
    true, false = cnt[1], cnt[0]
    return true / (false - true)

In [ ]:
asp_chance = np.array(list(map(get_chance, asp_bt)))
plac_chance = np.array(list(map(get_chance, plac_bt)))

In [ ]:
chance_ratio = plac_chance / asp_chance
chance_ratio.shape

In [ ]:
def stat_intervals(stat, alpha):
    return np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha) / 2])

[round(x, 4) for x in stat_intervals(chance_ratio, .05)]